<div style="background-color: RGB(0,0,0);background-color: RGB(0,0,0);">
    <h2 style="margin: auto; padding: 10px; color:#fff; text-align: center"></h2>
    <h1 style="margin: auto; padding: 10px; color:RGB(255,255,255); text-align: center">Législatives 2024 </h1>
    <h1 style="margin: auto; padding: 10px; color:#fff; text-align: center">Etude : Prévisions des intentions de votes </h1>
    <h2 style="margin: auto; padding: 10px; color:#fff; text-align: center">Premier Tour</h2>
    </div>  
<div style="text-align:center; background-color: RGB(0,0,0);">
    <img src="logo_elec.jpg" alt="billets" width="400" height="100"> 
</div>
<div style="background-color: RGB(0,0,0);background-color: RGB(0,0,0);">
    <h2 style="margin: auto; padding: 10px; color:#fff; text-align: left">Auteur: Stéphane Srsa</h2>
    </div> 

In [2]:
# Librairies et fonctions
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.features import GeoJson, GeoJsonTooltip
import geopandas as gpd
from branca.element import Element
import warnings
warnings.filterwarnings("ignore")
from Legis_Fonctions import *

<div style="background-color: RGB(0,0,0); margin-top: 10px; margin-bottom: 10px; border: 5px solid RGB(51,153,153);padding: 5px;" >
    <a name="C1" style="margin: auto; padding: 20px; line-height: 2.5; color:RGB(255,255,255); font-size: 20px; font-weight: bold;" >Analyse des résultats Européennes 2024 </a>
</div>

<div style="background-color: RGB(50,150,150); margin-top: 10px; margin-bottom: 10px; border: 5px solid RGB(0,0,0); padding: 5px;">
    <a name="C2" style="font-size: 20px; padding: 20px; font-family: 'Calibri'; color: RGB(255,255,255);"><strong>Préparation du fichier</strong></a>
</div>

In [5]:
# Charger les fichiers des résultats par circonscription et par commune:
Europ2024 = pd.read_csv('resultats-definitifs-par-circonscription.csv', sep=";")
# Renommer les colonnes pour correspondance au df 'gdf'
Europ2024.rename(columns={'Code circonscription législative': 'codeCirconscription',
                            'Libellé circonscription législative':'nomCirconscription','Libellé département':'nomDepartement'}, inplace=True)

In [6]:
Europ2024.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 322 columns):
 #    Column                      Dtype 
---   ------                      ----- 
 0    Code département            object
 1    nomDepartement              object
 2    codeCirconscription         object
 3    nomCirconscription          object
 4    Inscrits                    int64 
 5    Votants                     int64 
 6    % Votants                   object
 7    Abstentions                 int64 
 8    % Abstentions               object
 9    Exprimés                    int64 
 10   % Exprimés/inscrits         object
 11   % Exprimés/votants          object
 12   Blancs                      int64 
 13   % Blancs/inscrits           object
 14   % Blancs/votants            object
 15   Nuls                        int64 
 16   % Nuls/inscrits             object
 17   % Nuls/votants              object
 18   Numéro de panneau 1         int64 
 19   Nuance liste 1             

In [7]:
#--------------------------------------------- Mise en valeurs numériques des colonnes 'pourcentage' ---------------------------

# Colonnes récurrentes (par liste)
colonnes1 = [f'% Voix/exprimés {i}' for i in range(1, 39)]
colonnes2 = [f'% Voix/inscrits {i}' for i in range(1, 39)]
colonnes = colonnes1 + colonnes2

# Switch from 'object' to 'float64':
for col in colonnes:
    Europ2024[col] = Europ2024[col].str.replace(',', '.').str.rstrip('%').astype('float')

# Colonnes fixes
colonnes = [
    '% Votants', '% Abstentions', '% Exprimés/inscrits', '% Exprimés/votants', 
    '% Blancs/inscrits', '% Blancs/votants', '% Nuls/inscrits', '% Nuls/votants'
]
for col in colonnes:
    Europ2024[col] = Europ2024[col].str.replace(',', '.').str.rstrip('%').astype('float')
    
Europ2024.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 322 columns):
 #    Column                      Dtype  
---   ------                      -----  
 0    Code département            object 
 1    nomDepartement              object 
 2    codeCirconscription         object 
 3    nomCirconscription          object 
 4    Inscrits                    int64  
 5    Votants                     int64  
 6    % Votants                   float64
 7    Abstentions                 int64  
 8    % Abstentions               float64
 9    Exprimés                    int64  
 10   % Exprimés/inscrits         float64
 11   % Exprimés/votants          float64
 12   Blancs                      int64  
 13   % Blancs/inscrits           float64
 14   % Blancs/votants            float64
 15   Nuls                        int64  
 16   % Nuls/inscrits             float64
 17   % Nuls/votants              float64
 18   Numéro de panneau 1         int64  
 19   Nuance

<div style="background-color: RGB(50,150,150); margin-top: 10px; margin-bottom: 10px; border: 5px solid RGB(0,0,0); padding: 5px;">
    <a name="C2" style="font-size: 20px; padding: 20px; font-family: 'Calibri'; color: RGB(255,255,255);"><strong>Résultat des Elections Européennes
</strong></a>
</div>

In [9]:
# Sélection des colonnes utiles pour les listes de candidats et des voix
Selected_columns_nuance = [f'Libellé abrégé de liste {i}' for i in range(1, 39)]
Selected_columns_voix = [f'Voix {i}' for i in range(1, 39)]
Selected_columns_candidats = [f'Libellé de liste {i}' for i in range(1, 39)]
Selected_columns_vote = [f'% Voix/exprimés {i}' for i in range(1, 39)]
Selected_columns_vote_inscrit = [f'% Voix/inscrits {i}' for i in range(1, 39)]

# Extraction des données Circoncription
List_Candidats = Europ2024[Selected_columns_candidats].iloc[0].tolist()
Voix_Candidats = Europ2024[Selected_columns_voix].sum().tolist()
nuance_Candidats = Europ2024[Selected_columns_nuance].iloc[0].tolist()
Vote_Candidats = Europ2024[Selected_columns_vote].mean().tolist()
Vote_inscrits_Candidats = Europ2024[Selected_columns_vote_inscrit].mean().tolist()


# Création du DF suivant lesCirconcription
Df = pd.DataFrame({
    'Candidats': List_Candidats,
    'Nuance' : nuance_Candidats,
    'Nb de Voix': Voix_Candidats,
    '% voix_exprimées':Vote_Candidats,
    '% voix_inscrits':Vote_inscrits_Candidats
}, index=range(1, len(List_Candidats) + 1))

# Classement par nombre de voix
Df = Df.sort_values(by='Nb de Voix', ascending=False).round(2)

# Affichage des DataFrames
# Circoncription
Resultat_Circon = Df.reset_index()
Resultat_Circon.rename(columns={'index': 'No Liste'},inplace=True)
# Rappel fonction: Tmess(message, Color='firebrick', Align='center', Size='14', Police='arial', Weight='normal', Style='italic')
Tmess('Résultats Européennes 2024','firebrick', Size='20', Weight='bold')
display(Resultat_Circon)


,No Liste,Candidats,Nuance,Nb de Voix,% voix_exprimées,% voix_inscrits
0,5,LA FRANCE REVIENT ! AVEC JORDAN BARDELLA ET MA...,La FRANCE REVIENT,7765936,31.03,15.60
1,11,BESOIN D'EUROPE,BESOIN D'EUROPE,3614646,14.54,7.25
2,27,RÉVEILLER L'EUROPE,REVEIL EUR,3424216,13.52,6.90
3,4,LA FRANCE INSOUMISE - UNION POPULAIRE,LFI - UP,2448703,10.76,5.16
4,18,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FR...,1794171,7.12,3.63
5,6,EUROPE ÉCOLOGIE,EUROPE ÉCOLOGIE,1361883,5.45,2.74
6,3,"LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET ...","LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET ...",1353127,5.48,2.72
7,33,GAUCHE UNIE POUR LE MONDE DU TRAVAIL SOUTENUE ...,GAUCHE UNIE,584067,2.34,1.18
8,29,ALLIANCE RURALE,AR,582901,2.26,1.17
9,8,"PARTI ANIMALISTE - LES ANIMAUX COMPTENT, VOTRE...",PARTI ANIMALISTE,495936,1.95,1.00


<div style="background-color: RGB(50,150,150); margin-top: 10px; margin-bottom: 10px; border: 5px solid RGB(0,0,0); padding: 5px;">
    <a name="C2" style="font-size: 20px; padding: 20px; font-family: 'Calibri'; color: RGB(255,255,255);"><strong>Listes arrivées en-tête par Circonscription</strong></a>
</div>

In [11]:
# Classement des Nb_Lists premières :
EuropCirco = TopList_Europ(Europ2024, Nb_Lists = 20)

# Création de la colonne 'somme' en additionnant les valeurs des colonnes numériques
# Sélection des colonnes numériques utilisant une expression régulière
colonnes_numeriques = [col for col in EuropCirco.columns if col.startswith('% Voix Liste')]
EuropCirco['%TotalVotes'] =EuropCirco[colonnes_numeriques].sum(axis=1)
EuropCirco = EuropCirco.sort_values(by="%TotalVotes", ascending=False)

display(EuropCirco.head(2))
display(EuropCirco.tail(2))

,Code département,nomDepartement,codeCirconscription,nomCirconscription,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Liste17,% Voix Liste17,Liste18,% Voix Liste18,Liste19,% Voix Liste19,Liste20,% Voix Liste20,somme,%TotalVotes
556,975,Saint-Pierre-et-Miquelon,97501,Saint-Pierre-et-Miquelon,5065,1071,21.15,3994,78.85,1023,...,FRANCE LIBRE,0.10,HUMANITE SOUVERAINE,0.00,POUR UNE DEMOCRATIE REELLE : DECIDONS NOUS-MEM...,0.00,FREE PALESTINE,0.0,100.02,100.02
561,987,Polynésie française,98702,2ème circonscription,69181,7399,10.70,61782,89.30,7103,...,NLP,0.03,PARTI PIRATE,0.01,DEFENDRE LES ENFANTS,0.01,HUMANITE SOUVERAINE,0.0,100.01,100.01


,Code département,nomDepartement,codeCirconscription,nomCirconscription,Inscrits,Votants,% Votants,Abstentions,% Abstentions,Exprimés,...,Liste17,% Voix Liste17,Liste18,% Voix Liste18,Liste19,% Voix Liste19,Liste20,% Voix Liste20,somme,%TotalVotes
504,92,Hauts-de-Seine,9212,12ème circonscription,94555,55899,59.12,38656,40.88,54909,...,ÉQUINOXE,0.38,FORTERESSE EUROPE,0.29,EUROPE TERRITOIRES ÉCOLOGIE,0.28,PACE,0.27,99.02,99.02
548,973,Guyane,97302,2ème circonscription,49813,3722,7.47,46091,92.53,3528,...,PARTI ANIMALISTE,0.17,PARTI PIRATE,0.17,FRANCE LIBRE,0.17,PPL,0.14,99.02,99.02


<div style="background-color: RGB(0,0,0); margin-top: 10px; margin-bottom: 10px; border: 5px solid RGB(51,153,153);padding: 5px;" >
    <a name="C1" style="margin: auto; padding: 20px; line-height: 2.5; color:RGB(255,255,255); font-size: 20px; font-weight: bold;" >Analyse des résultats LEGISLATIVES 2022 - Listes arrivées en-tête par Circonscription</a>
</div>

In [13]:
Legis2022 = pd.read_excel("Legis2022-Circons.xlsx")
Legis2022.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   COD_CIRCONS     550 non-null    object 
 1   Inscrits        550 non-null    int64  
 2   Abstentions     550 non-null    int64  
 3    % Abs/Ins      550 non-null    float64
 4   Votants         550 non-null    int64  
 5   % Vot/Ins       550 non-null    float64
 6   Blancs          550 non-null    int64  
 7   Blancs/Ins      550 non-null    float64
 8   % Blancs/Vot    550 non-null    float64
 9   Nuls            550 non-null    int64  
 10  % Nuls/Ins      550 non-null    float64
 11  % Nuls/Vot      550 non-null    float64
 12  Exprimés        550 non-null    int64  
 13  % Exp/Ins       550 non-null    float64
 14  % Exp/Vot       550 non-null    float64
 15  DIV Voix        550 non-null    int64  
 16  DIV % Voix/Exp  550 non-null    float64
 17  DSV Voix        550 non-null    int

In [17]:
Resultat_2022 = top_Legis2022(Legis2022,5)
Resultat_2022

,COD_CIRCONS,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,Blancs/Ins,% Blancs/Vot,Nuls,...,Top_1_List,Top_1_%,Top_2_List,Top_2_%,Top_3_List,Top_3_%,Top_4_List,Top_4_%,Top_5_List,Top_5_%
0,0101,86187,43652,49.306930,42535,50.693070,490,0.586842,1.165789,234,...,LR % Voix/Exp,25.43,NUP % Voix/Exp,23.01,RN % Voix/Exp,22.94,ENS % Voix/Exp,18.42,REC % Voix/Exp,4.67
1,0102,98818,49731,49.657596,49087,50.342500,619,0.650673,1.292692,166,...,ENS % Voix/Exp,25.77,NUP % Voix/Exp,24.87,RN % Voix/Exp,23.75,LR % Voix/Exp,15.32,REC % Voix/Exp,4.85
2,0103,82204,45946,52.908636,36258,47.091364,385,0.647636,1.332182,144,...,ENS % Voix/Exp,28.19,NUP % Voix/Exp,23.07,LR % Voix/Exp,17.92,RN % Voix/Exp,15.59,REC % Voix/Exp,5.03
3,0104,94897,49258,50.662672,45639,49.337414,640,0.736810,1.499914,207,...,RN % Voix/Exp,25.52,LR % Voix/Exp,16.89,NUP % Voix/Exp,16.89,ENS % Voix/Exp,15.72,DVC % Voix/Exp,8.93
4,0105,77456,40382,47.157338,37074,52.842662,820,1.375468,2.502662,206,...,NUP % Voix/Exp,24.32,RN % Voix/Exp,19.64,DVD % Voix/Exp,18.32,LR % Voix/Exp,9.18,REC % Voix/Exp,5.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,9707,238760,168793,71.126593,69967,28.873462,1359,0.900824,3.273187,1307,...,NUP % Voix/Exp,24.94,DVC % Voix/Exp,12.13,ENS % Voix/Exp,10.72,DVG % Voix/Exp,8.68,RN % Voix/Exp,7.68
546,9708,131219,115176,83.964167,16043,16.035833,180,0.245000,1.565833,16,...,ENS % Voix/Exp,27.31,UDI % Voix/Exp,23.15,NUP % Voix/Exp,22.74,REC % Voix/Exp,7.05,DIV % Voix/Exp,3.30
547,9709,120556,102826,84.742778,17730,15.257222,392,0.357222,2.686111,70,...,NUP % Voix/Exp,40.20,ENS % Voix/Exp,25.91,DVD % Voix/Exp,6.38,REC % Voix/Exp,5.81,LR % Voix/Exp,4.59
548,9710,104303,83943,78.094894,20360,21.905106,290,0.407021,1.868511,45,...,NUP % Voix/Exp,33.08,ENS % Voix/Exp,28.05,LR % Voix/Exp,9.35,REC % Voix/Exp,8.07,DVC % Voix/Exp,3.50
